In [1]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
from tensorflow import keras
from typing import Dict, Text

2022-10-13 22:44:46.517386: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-13 22:44:46.845468: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-13 22:44:46.976547: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-13 22:44:48.107149: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
ratings = tfds.load("movielens/1m-ratings", split="train")
movies = tfds.load("movielens/1m-movies", split="train")



2022-10-13 22:44:50.170545: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 83.05 url/s] 
Extraction completed...: 0 file [00:00, ? file/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 52.95 url/s]
2022-10-13 23:02:52.304905: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-13 23:02:52.304933: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tyo-Katana-GF66-11UC): /proc/driver/nvidia/version does not exist
2022-10-13 23:02:52.306623: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Dataset movielens downloaded and prepared to ~/tensorflow_datasets/movielens/1m-ratings/0.1.1. Subsequent calls will reuse this data.


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 94.60 url/s] 
Extraction completed...: 0 file [00:00, ? file/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 63.40 url/s]


Dataset movielens downloaded and prepared to ~/tensorflow_datasets/movielens/1m-movies/0.1.1. Subsequent calls will reuse this data.


In [3]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
ratings

movies = movies.map(lambda x: x["movie_title"])

In [4]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [5]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie = np.unique(np.concatenate(list(movie_titles)))
unique_ids = np.unique(np.concatenate(list(user_ids)))
unique_movie[:10]

array([b'$1,000,000 Duck (1971)', b"'Night Mother (1986)",
       b"'Til There Was You (1997)", b"'burbs, The (1989)",
       b'...And Justice for All (1979)', b'1-900 (1994)',
       b'10 Things I Hate About You (1999)', b'101 Dalmatians (1961)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)'], dtype=object)

In [6]:
model1 = tf.keras.Sequential([
  tf.keras.layers.StringLookup(vocabulary=unique_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_ids) + 1, 32),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128)
])



In [7]:
model2 = tf.keras.Sequential([
    tf.keras.layers.StringLookup(vocabulary=unique_movie, mask_token=None),
    tf.keras.layers.Embedding(len(unique_movie) + 1, 32),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128)
])

In [8]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(model2),
  ks = (200, 400, 600, 800, 1000)
)


In [9]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [10]:
class MovielensModel(tfrs.Model):

  def __init__(self, model1, model2):
    super().__init__()
    self.model2: tf.keras.Model = model2
    self.model1: tf.keras.Model = model1
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.model1(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.model2(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [11]:
class NoBaseClassMovielensModel(tf.keras.Model):

  def __init__(self, model1, model2):
    super().__init__()
    self.model2: tf.keras.Model = model2
    self.model1: tf.keras.Model = model1
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    with tf.GradientTape() as tape:

      # Loss computation.
      user_embeddings = self.model1(features["user_id"])
      positive_movie_embeddings = self.model2(features["movie_title"])
      loss = self.task(user_embeddings, positive_movie_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Loss computation.
    user_embeddings = self.model1(features["user_id"])
    positive_movie_embeddings = self.model2(features["movie_title"])
    loss = self.task(user_embeddings, positive_movie_embeddings)

    # Handle regularization losses as well.
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

In [12]:
model_1 = MovielensModel(model1, model2)
model_1.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [13]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [14]:
model_1.fit(cached_train, epochs=100)

Epoch 1/100
10/10 [==============================] - 36s 3s/step - factorized_top_k/top_200_categorical_accuracy: 0.0778 - factorized_top_k/top_400_categorical_accuracy: 0.1499 - factorized_top_k/top_600_categorical_accuracy: 0.2148 - factorized_top_k/top_800_categorical_accuracy: 0.2742 - factorized_top_k/top_1000_categorical_accuracy: 0.3237 - loss: 72589.2969 - regularization_loss: 0.0000e+00 - total_loss: 72589.2969
Epoch 2/100
10/10 [==============================] - 32s 3s/step - factorized_top_k/top_200_categorical_accuracy: 0.1106 - factorized_top_k/top_400_categorical_accuracy: 0.1957 - factorized_top_k/top_600_categorical_accuracy: 0.2714 - factorized_top_k/top_800_categorical_accuracy: 0.3403 - factorized_top_k/top_1000_categorical_accuracy: 0.4022 - loss: 70232.3331 - regularization_loss: 0.0000e+00 - total_loss: 70232.3331
Epoch 3/100
10/10 [==============================] - 31s 3s/step - factorized_top_k/top_200_categorical_accuracy: 0.1505 - factorized_top_k/top_400_cate

In [15]:
ratings = tfds.load("movielens/1m-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

In [16]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [17]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [18]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [19]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [20]:
class MovielensRate(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [21]:
model_2 = MovielensRate()
model_2.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [22]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [23]:
model_2.fit(cached_train, epochs=1000)

Epoch 1/1000
10/10 [==============================] - 3s 28ms/step - root_mean_squared_error: 2.0875 - loss: 4.0404 - regularization_loss: 0.0000e+00 - total_loss: 4.0404
Epoch 2/1000
10/10 [==============================] - 0s 12ms/step - root_mean_squared_error: 1.1708 - loss: 1.3623 - regularization_loss: 0.0000e+00 - total_loss: 1.3623
Epoch 3/1000
10/10 [==============================] - 0s 12ms/step - root_mean_squared_error: 1.1166 - loss: 1.2486 - regularization_loss: 0.0000e+00 - total_loss: 1.2486
Epoch 4/1000
10/10 [==============================] - 0s 11ms/step - root_mean_squared_error: 1.1091 - loss: 1.2326 - regularization_loss: 0.0000e+00 - total_loss: 1.2326
Epoch 5/1000
10/10 [==============================] - 0s 11ms/step - root_mean_squared_error: 1.1049 - loss: 1.2232 - regularization_loss: 0.0000e+00 - total_loss: 1.2232
Epoch 6/1000
10/10 [==============================] - 0s 11ms/step - root_mean_squared_error: 1.0996 - loss: 1.2110 - regularization_loss: 0.0000

In [24]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [25]:
index = tfrs.layers.factorized_top_k.BruteForce(model_1.model1)

index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model_1.model2)))
)
id = 200
_, titles = index(tf.constant([f'{id}']))
print(f"Recommendations for user {id} : {titles[0, :100]}\n")
list_title = list(titles.numpy().astype('str'))
res = [list(item) for item in list_title]
flat_l = flatten_list(res)
test_ratings = {}
test_movie_titles = flat_l
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model_2({
      "user_id": np.array([f"{id}"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Recommendations for user 200 : [b'Entrapment (1999)' b'Arachnophobia (1990)'
 b'Austin Powers: The Spy Who Shagged Me (1999)'
 b'Wedding Singer, The (1998)' b'Cruel Intentions (1999)'
 b'Maverick (1994)' b'Me, Myself and Irene (2000)'
 b'Batman Forever (1995)' b'Pleasantville (1998)'
 b'Green Mile, The (1999)']

Ratings:
Arachnophobia (1990): [[3.9805682]]
Green Mile, The (1999): [[3.6274908]]
Maverick (1994): [[3.58293]]
Wedding Singer, The (1998): [[3.2172897]]
Pleasantville (1998): [[2.9695745]]
Austin Powers: The Spy Who Shagged Me (1999): [[2.955096]]
Cruel Intentions (1999): [[2.72666]]
Me, Myself and Irene (2000): [[2.586542]]
Entrapment (1999): [[1.9506471]]
Batman Forever (1995): [[1.8232152]]
